<a href="https://colab.research.google.com/github/vrra/FGAN-Build-a-thon/blob/main/Notebooks2023/Read-semi-annotated-push-to-argilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created: 1 Jan 2024 (Happy New Year!!)

Aaron, Othniel, Israel, Prof. James, Vishnu.

This notebook adds records to the dataset.

Pre-requisites:

the following notebooks are already run:

1. Create the raw dataset in HF hub.

2. Configure the argilla dataset

this notebook will

add records in the argilla dataset from the raw dataset

and later you need to

##annotate the dataset in UI

## Install Libraries

Install the latest version of Argilla in Colab, along with other libraries and models used in this notebook.

In [ ]:
!pip install argilla datasets pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00


Prerequisites

Deploy Argilla Server on [HF Spaces](https://huggingface.co/new-space?template=argilla/argilla-template-space).


More info on Installation [here](../getting_started/installation/deployments/deployments.html).

## Secretes needed




* `ARGILLA_API_URL`: It is the url of the Argilla Server.
  * If you're using HF Spaces, it is constructed as `https://[your-owner-name]-[your_space_name].hf.space`.
* `ARGILLA_API_KEY`: It is the API key of the Argilla Server. It is `owner` by default.
* `HF_TOKEN`: It is the Hugging Face API token. It is only needed if you're using a [private HF Space](https://docs.argilla.io/en/latest/getting_started/installation/deployments/huggingface-spaces.html#deploy-argilla-on-spaces). You can configure it in your profile: [Setting > Access Tokens](https://huggingface.co/settings/tokens).
* `workspace`: admin


In [ ]:
import argilla as rg
from argilla._constants import DEFAULT_API_KEY

In [ ]:
from google.colab import userdata
api_url= userdata.get('my_argilla_url')
api_key= userdata.get('my_argilla_key')

import argilla as rg
rg.init(api_url=api_url, api_key=api_key)

# # If you want to use your private HF Space
# rg.init(extra_headers={"Authorization": f"Bearer {hf_token}"})

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:168: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(


In [ ]:
from datasets import load_dataset
import pandas as pd

# Load and inspect a raw dataset from the Hugging Face Hub
# (and not the pre-processed or annotated dataset in the spaces).
# ITU-AN-usecases-1 is the dataset name in HF hub (and not the spaces name)
hf_dataset = load_dataset('vishnuramov/ITU-AN-usecases-1')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['page_number', 'document_source', 'page_content'],
        num_rows: 98
    })
})

In [ ]:
df = pd.DataFrame(hf_dataset)


In [ ]:
df['train'][0]['page_content']

'I n t e r n a t i o n a l  T e l e c o m m u n i c a t i o n  U n i o n  \n  \nITU-T  Technical Specification  \nTELECOMMUNICATION  \nSTANDARDIZATION SECTOR  \nOF ITU   \n(28 October  2021 ) \n \nITU-T Focus Group on Autonomous Networks  \n Technical Specification  \nUse cases for Autonomous Networks'

In [ ]:
#whereas this one is a spaces dataset,
#for annotation with argilla, deployed in HF spaces
#(and not the raw dataset in the HF hub)
dataset = rg.FeedbackDataset.from_argilla(name="my-first-dataset", workspace="admin")
records = [
    rg.FeedbackRecord(
        fields={"prompt": record["page_content"],
                "context": record['document_source']+
                            " page number= "+
                            str(record['page_number'])}
    )
    for record in hf_dataset['train']
    ]




In [ ]:
dataset.add_records(records)

Output()

CAUTION: all records will get updated

In [ ]:
# Load the dataset
modified_records = []
# Loop through the records and make modifications
for record in dataset.records:
    # e.g. adding/modifying a metadata field and vectors
    record.metadata["my_metadata"] = "new_metadata"
    # e.g. removing all suggestions and responses
    record.suggestions = []
    record.responses = []
    modified_records.append(record)

dataset.update_records(modified_records)

Output()

CAUTION: proceed with caution

deleting the records.

In [ ]:
# List the records to be deleted
numRecords = len(dataset.records)
records_to_delete = list(dataset.records[:numRecords])
# Delete the list of records from the dataset
dataset.delete_records(records_to_delete)

-------------

